In [ ]:
import torch 
import torch.nn as nn

import numpy as np
import random as rd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas as pd
import pickle as pkl

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import torch 
import torch.nn as nn

import numpy as np
import random as rd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas as pd

from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# with open("predictions/basic/eval_fnv_with_BERT_FT_2_epoch_5.pkl", "rb") as file:
#     fichier = torch.load(file, map_location=torch.device('cpu'))

with open("predictions/basic/saved_with_pickle", "rb") as file:
    fichier = torch.load(file, map_location=torch.device('cpu'))
    
print(fichier)

def plot_residuals(y_true, predictions, matplotlib=True, Plotly=False, target_normalizer=True):
    
    if target_normalizer is not None:
        predictions = target_normalizer.inverse_transform(torch.cat(predictions, dim=0).cpu().numpy())
        y_true = target_normalizer.inverse_transform(torch.cat(y_true, dim=0).cpu().numpy())
    else:
        predictions = torch.cat(predictions, dim=0).cpu().numpy()
        y_true = torch.cat(y_true, dim=0).cpu().numpy()

    fig, ax = plt.subplots(len(target_names), 1, figsize=(6,33))
    
    for i, target in enumerate(target_names):
        
        Y_HAT = predictions[:, i]
        Y_TRUE = y_true[:, i]
        
        residuals = Y_TRUE - Y_HAT
        
        if matplotlib:
            plt.figure(figsize=(5, 5))
            plt.subplot(len(target_names), 1, i+1)
            plt.scatter(Y_TRUE, residuals, color='blue', alpha=0.7, label="Résidus")
            plt.axhline(0, color='red', linestyle='-', linewidth=1)  # Ligne y=0
            plt.xlabel("Prédictions")
            plt.ylabel("Résidus")
            plt.grid(linewidth=0.3, alpha=0.3, color='#000000')
            plt.title(f'Residuals {target}')
            plt.legend()
            plt.tight_layout()
            plt.show()

        if Plotly:
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=Y_TRUE, y=residuals, mode='markers', marker=dict(color='blue'), name="Résidus"))
            fig.add_trace(go.Scatter(x=[min(Y_TRUE)-10, max(Y_TRUE)+10], y=[0, 0], mode='lines', line=dict(color='red', dash='solid'), name="y=0"))
            fig.update_layout(title=f"Residuals {target}", xaxis_title="Prédictions", yaxis_title="Résidus")
            fig.show()

# Appeler la fonction avec matplotlib et Plotly activés
plot_residuals(y_true, predictions, matplotlib=True, Plotly=False)


def Plot_errors_models(predictions=None, true_targets=None, target_names=None, target_normalizer=None, save=True, save_path="./model_test.pdf"):

    sns.set(style="whitegrid")

    # Création d'exemple de données
    modeles = [f'Modèle {i}' for i in range(1, 5)]
    chemicals = [f'Chimique {i}' for i in range(1, 7)]

    # Construction du DataFrame
    data = {
        'Modèle': [],
        'Chimique': [],
        'Erreur': []
    }

    for modele in modeles:
        for chim in chemicals:
            error = np.random.rand()  # génère une erreur entre 0 et 1 (exemple)
            data['Modèle'].append(modele)
            data['Chimique'].append(chim)
            data['Erreur'].append(error)

    df = pd.DataFrame(data)

    # Création du graphique avec Seaborn via catplot
    g = sns.catplot(
        data=df,
        x='Modèle',
        y='Erreur',
        hue='Chimique',
        kind='bar',
        height=6,
        aspect=1.8,
        palette="hls",
        dodge=True  # Permet un léger espacement entre les groupes
    )

    # Réduction de la largeur des barres pour ajouter un petit espacement
    for bars in g.ax.containers:
        for bar in bars:
            bar.set_width(bar.get_width() * 0.9)  # Réduction de la largeur des barres

    # Personnalisation des axes et du titre
    g.set_axis_labels("Modèle", "Erreur")
    g.fig.suptitle("Erreur par modèle et valeur chimique", y=1.03)

    # Modification de la limite inférieure de l'axe des ordonnées (y)
    g.set(ylim=(-0.1, None))

    # Ajout d'une ligne horizontale en pointillé sur y=0, en noir et plus fine
    g.ax.axhline(0, color='red', linestyle='--', linewidth=0.7)

    # Ajout des points de suspension sous chaque barre
    for bars in g.ax.containers:
        for bar in bars:
            height = bar.get_height()
            g.ax.text(bar.get_x() + bar.get_width() / 2, -0.02, "UFV", ha='center', va='top', fontsize=9, color='black', rotation=0)

    plt.show()

Plot_errors_models()


def show_predictions(predictions, true_targets, target_names, target_normalizer=None, matplot=True, seabor=False, save=True, save_path="./model_test.pdf"):
    
    if target_normalizer is not None:
        y_hat = target_normalizer.inverse_transform(torch.cat(predictions, dim=0).cpu().numpy())
        y_true = target_normalizer.inverse_transform(torch.cat(true_targets, dim=0).cpu().numpy())
    else:
        y_hat = torch.cat(predictions, dim=0).cpu().numpy()
        y_true = torch.cat(true_targets, dim=0).cpu().numpy()

    Y_HAT = y_hat[:,i]
    Y_TRUE = y_true[:,i]
    r2 = r2_score(Y_TRUE, Y_HAT)
    rmse = root_mean_squared_error(Y_TRUE, Y_HAT)
    mae = mean_absolute_error(Y_TRUE, Y_HAT)
    mape = mean_absolute_percentage_error(Y_TRUE, Y_HAT)
            
    if matplot:
        fig, ax = plt.subplots(len(target_names),1, figsize=(6,33))
        for i, target in enumerate(target_names):
            plt.subplot(len(target_names), 1, i+1)
            sns.scatterplot(x=Y_HAT, y=Y_TRUE, 
                    palette='viridis', 
                    hue=np.abs(Y_HAT - Y_TRUE))
            sns.lineplot(x=Y_TRUE, y=Y_TRUE)
            plt.title(f'Model Predictions vs Ground Truth for {target}\nRMSE: {rmse:.4f}, R2: {r2:.4f}, MAE: {mae:.4f}, MAPE : {mape:.4f}')
            plt.xlabel(f'Predicted {target}')
            plt.ylabel(f'True {target}')
        if save:
            plt.savefig(save_path, format='pdf', bbox_inches='tight')
        plt.show()
    
    if seabor:
        for i, target in enumerate(target_names):
            Y_HAT = y_hat[:, i]
            Y_TRUE = y_true[:, i]
            df_plot = pd.DataFrame({f'Predicted {target}': Y_HAT, f'True {target}': Y_TRUE})
            fig = px.scatter(df_plot, x=f'Predicted {target}', y=f'True {target}', 
                             color=np.abs(Y_HAT - Y_TRUE), 
                             title=f'Model Predictions vs Ground Truth for {target}',
                             color_continuous_scale='viridis')
            fig.add_shape(type="line", x0=min(Y_TRUE), y0=min(Y_TRUE), x1=max(Y_TRUE), y1=max(Y_TRUE),
                          line=dict(color="red", width=2, dash="dash"))
            fig.show()
       

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.